# ライブラリの読み込み

In [1]:
# 警告を消す
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
import optuna
import lightgbm as lgb

# データセット

In [2]:
submit = pd.read_csv('submit_sample.csv',header=None)
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

# データの確認

In [3]:
train.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,31,services,married,secondary,no,12294,yes,no,cellular,21,nov,101,3,498,0,other,0
1,1,29,entrepreneur,single,tertiary,no,43027,no,no,cellular,22,aug,158,2,702,0,unknown,1
2,2,35,management,married,tertiary,no,12252,yes,no,cellular,11,nov,351,1,826,0,failure,0
3,3,31,technician,married,secondary,no,99121,yes,yes,unknown,16,may,658,2,120,0,failure,0
4,4,48,unemployed,married,primary,no,42005,yes,no,telephone,3,apr,177,1,273,0,unknown,0


In [4]:
# 欠損値の確認
train.isnull().sum()

id           0
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [5]:
# カテゴリー数の確認
train.nunique()

id           27100
age             42
job             11
marital          3
education        4
default          2
balance      23967
housing          2
loan             2
contact          3
day             30
month           11
duration       140
campaign         5
pdays          872
previous         4
poutcome         4
y                2
dtype: int64

In [6]:
# 特定のカテゴリーにどのくらい値があるのかを確認する。
train['poutcome'].value_counts()

unknown    23099
failure     2717
other        826
success      458
Name: poutcome, dtype: int64

# データの結合　処理

In [7]:
# データの結合（trainとtestデータそれぞれに前処理をかけるため結合しておく。）
data = pd.concat([train, test], sort = False)

In [8]:
# それぞれlightgbmで扱えるようにカテゴリを数値に変換しておく
# 数の少ない変換は以下のように直接変換を行うこともできる。
data['month'].replace(['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'], [1,2,3,4,5,6,7,8,9,10,11,12], inplace=True)
data['housing'].replace(['yes','no'], [1, 0], inplace= True)
data['loan'].replace(['yes','no'], [1, 0], inplace= True)
data['default'].replace(['yes','no'], [1, 0], inplace= True)

In [9]:
# LabelEncoderによるLable化
# 機械学習モデルで扱うために文字列を数値型に変換していく。
cat_features = ['job','marital', 'education', 'contact', 'poutcome']
for col in cat_features:
    lbl = LabelEncoder()
    data[col] = lbl.fit_transform(data[col].values)
    
data.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,31,7,1,1,0,12294,1,0,0,21,11,101,3,498,0,1,0.0
1,1,29,2,2,2,0,43027,0,0,0,22,8,158,2,702,0,3,1.0
2,2,35,4,1,2,0,12252,1,0,0,11,11,351,1,826,0,0,0.0
3,3,31,9,1,1,0,99121,1,1,2,16,5,658,2,120,0,0,0.0
4,4,48,10,1,0,0,42005,1,0,1,3,4,177,1,273,0,3,0.0


# 機械学習モデルに渡すためのデータを用意する。

In [10]:
train = data[:len(train)]
test = data[len(train):]

In [11]:
y_train = train['y']
X_train = train.drop(['y','id','pdays'], axis=1)
X_test = test.drop(['y','id','pdays'], axis=1)

In [12]:
# カテゴリー変数の宣言
categorical_features = ['job', 'marital', 'education','housing','loan','contact','month','poutcome']

# パラメーターチューニング

In [13]:
#  optunaを使用した自動パラメーターチューニング

# パラメーターチューニング用のデータをセット
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features, free_raw_data=False)
lgb_test = lgb.Dataset(X_test)

# metric auc、objective binary 固定
params = {
    'metric' :'auc',
    "objective": "binary",
}

# 複数のパラメータの条件で検証を行い最適の（もっとも評価が高くなる）パラメータをbest_paramsとし代入する。
clf = optuna.integration.lightgbm.train(
    params, lgb_train, 
    valid_sets=lgb_test, 
    verbose_eval=10, 
    early_stopping_rounds=5,
 )

best_params = clf.params

feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 1


feature_fraction, val_score: 1.000000:  14%|#4        | 1/7 [00:00<00:01,  5.59it/s]

Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 1


feature_fraction, val_score: 1.000000:  29%|##8       | 2/7 [00:00<00:00,  5.59it/s]

...

min_data_in_leaf, val_score: 1.000000:  60%|######    | 3/5 [00:00<00:00,  4.27it/s]

Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 1


min_data_in_leaf, val_score: 1.000000:  80%|########  | 4/5 [00:01<00:00,  4.42it/s]

Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 1


min_data_in_leaf, val_score: 1.000000: 100%|##########| 5/5 [00:01<00:00,  4.55it/s]


In [14]:
best_params

{'metric': 'auc',
 'objective': 'binary',
 'lambda_l1': 0.0,
 'lambda_l2': 0.0,
 'num_leaves': 31,
 'feature_fraction': 1.0,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20}

# バリデーション

In [15]:
# 関数を定義
# 評価指標はROC
def get_evaluate(y_test, predict):
    fpr, tpr, thr_arr = metrics.roc_curve(y_test, predict)
    accuracy = metrics.auc(fpr, tpr)
    roc = metrics.roc_auc_score(y_test, predict)      
    return roc

In [16]:
# KFoldクラスを用いて分割 
cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=0)
roc_list = []

for train_index, test_index in cv.split(X_train, y_train):
    X_tr = X_train.iloc[train_index]
    y_tr = y_train.iloc[train_index]
    X_va = X_train.iloc[test_index]
    y_va = y_train.iloc[test_index]
    lgb_train = lgb.Dataset(X_tr, y_tr, categorical_feature=categorical_features)
    lgb_test = lgb.Dataset(X_test)
    
#     パラメーターチューニングで作成したbest_paramsを利用する。
    clf = lgb.train(best_params, lgb_train,100)
    y_predict = clf.predict(X_va, num_iteration=clf.best_iteration)
    roc= get_evaluate(y_va, y_predict)
    print('roc:{}'.format(roc))
    roc_list.append(roc)
print('Kfold平均 roc:{}'.format(np.mean(roc_list)))
y_predict

roc:0.8515933102434623
roc:0.8514929590445838
roc:0.8380754580380401
roc:0.8451371809983944
Kfold平均 roc:0.8465747270811201


array([0.00697243, 0.00103702, 0.06979217, ..., 0.03946388, 0.14313303,
       0.01468361])

# lightgbmによる予測モデル作成

In [17]:
# train_test_splitによるデータ分割
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0, stratify=y_train)

lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_features)

params = {
    'metric' :'auc',
    'objective': 'binary'
}

model = lgb.train(
    best_params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=10,
    num_boost_round=1000,
    early_stopping_rounds=10
)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)


Training until validation scores don't improve for 10 rounds
[10]	training's auc: 0.855182	valid_1's auc: 0.820093
[20]	training's auc: 0.882354	valid_1's auc: 0.837667
[30]	training's auc: 0.898999	valid_1's auc: 0.844043
[40]	training's auc: 0.912523	valid_1's auc: 0.846599
[50]	training's auc: 0.923456	valid_1's auc: 0.845881
Early stopping, best iteration is:
[42]	training's auc: 0.914538	valid_1's auc: 0.847076


# 予測値を提出する

In [18]:
submit = submit.drop(1, axis=1)
submit['1'] = y_pred

In [19]:
submit

,0,1
0,0,0.742985
1,1,0.067566
2,2,0.028508
3,3,0.003566
4,4,0.058677
...,...,...
18045,18045,0.020304
18046,18046,0.006221
18047,18047,0.067945
18048,18048,0.007265
